# IBM Applied Data Science Capstone Course by Coursera
## Week 3 Part 1

### Build a dataframe of the postal code of each neighborhood along with the borough name and neighborhood name in Toronto.

### **Step 1:** Load Required Libraries

In [85]:
import numpy as np # library to handle data in a vectorized manner

import pandas as pd # library for data analsysis
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import json # library to handle JSON files

#!conda install -c conda-forge geopy --yes # uncomment this line if you haven't completed the Foursquare API lab
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

from bs4 import BeautifulSoup

### Step 2: Scrape the Wikipedia Page with Beautiful Soup

In [86]:
source = requests.get("https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M").text
soup = BeautifulSoup(source,'lxml')

### Step 3: Create New Lists to store the Table Data, then find and append the correct table data into each list

In [87]:
# Create lists to store the table data
postalCodeList = []
boroughList = []
neighborhoodList = []

In [88]:
# Append the data into the respective lists
for row in soup.find('table').find_all('tr'):
    cells = row.find_all('td')
    if(len(cells) > 0):
        postalCodeList.append(cells[0].text)
        boroughList.append(cells[1].text)
        neighborhoodList.append(cells[2].text.rstrip('\n')) # avoid new lines in neighborhood cell

### Step 4: Create the DataFrame

In [89]:
# Create a new DataFrame using the three lists just populated
TorontoNeighborhoods = pd.DataFrame({"PostalCode": postalCodeList,
                           "Borough": boroughList,
                           "Neighborhood": neighborhoodList})

TorontoNeighborhoods.head()

,PostalCode,Borough,Neighborhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Harbourfront


### Step 5: Drop Boroughs that are "Not Assigned"

In [94]:
# Drop cells with a borough that is Not assigned
TorontoNeighborhoods_update1 = TorontoNeighborhoods[TorontoNeighborhoods.Borough != "Not assigned"].reset_index(drop=True)
TorontoNeighborhoods_update1.head()

,PostalCode,Borough,Neighborhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,Harbourfront
3,M5A,Downtown Toronto,Regent Park
4,M6A,North York,Lawrence Heights


### Step 6: Group Neighborhoods with the same Postal Code

In [95]:
# group neighborhoods in the same postal code
TorontoNeighborhoods_cleaned = TorontoNeighborhoods_update1.groupby(["PostalCode", "Borough"], as_index=False).agg(lambda x: ", ".join(x))
TorontoNeighborhoods_cleaned.head()

,PostalCode,Borough,Neighborhood
0,M1B,Scarborough,"Rouge, Malvern"
1,M1C,Scarborough,"Highland Creek, Rouge Hill, Port Union"
2,M1E,Scarborough,"Guildwood, Morningside, West Hill"
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae


### Step 7: Populate any "Not assigned" Neighborhood with its Borough

In [96]:
# for any Neighborhood that is "Not assigned" make the value the same as Borough
for index, row in TorontoNeighborhoods_cleaned.iterrows():
    if row["Neighborhood"] == "Not assigned":
        row["Neighborhood"] = row["Borough"]
        
TorontoNeighborhoods_cleaned.head()

,PostalCode,Borough,Neighborhood
0,M1B,Scarborough,"Rouge, Malvern"
1,M1C,Scarborough,"Highland Creek, Rouge Hill, Port Union"
2,M1E,Scarborough,"Guildwood, Morningside, West Hill"
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae


### Step 8: Confirm the Shape of the dataframe

In [97]:
TorontoNeighborhoods_cleaned.shape

(103, 3)